# Capstone Project - Clustering and Analysis of New York city neighborhoods 


#### Applied Data Science Capstone by IBM/Coursera


####  Douglas Massuia

## Table of contents:


[1) Introduction](#introduction)

[2) Data](#data)

[3) Methodology](#methodology)

[4) Results and Discussion](#results)

[5) Conclusion](#conclusion)

## 1) Introduction:   <a name="introduction"></a>

### 1.1) Business Problem

How does New York city neighborhoods correlate to each other and which ones are similar but not necessarily close by physically? How can we group them by venue types and why does that matter? 

In this project we will direct our efforts on detecting connections in New York city neighborhoods that aren't visible if you just look at their actual location and coordinate. We want to discover which neighborhoods are related to others based on their venue types, not necessarily those that are close to each other, and how they are spread over the Big Apple.

### 1.2) Interest

Clustering is considered to be one of the most popular unsupervised machine learning techniques used for grouping data points, or objects that are somehow similar. Cluster analysis has many applications in different domains, whether it be a bank's desire to segment his customers based on certain characteristics, or helping an individual to organize in-group his, or her favorite types of music. Generally speaking though, clustering is used mostly for discovering structure, summarization, and anomaly detection.

This application/algorithm would be able to help solve real world problems like:

* Provide business a tool that would allow them to choose a location to open a new franchise based on their current customer profile segmentation

* Create a customer profile based on their surroundings, or improve your current profiles with more data, better understanding their lifestyle and inclinations

* Provide the public administration sector a tool that would allow them to more deeply understand NYC venues distribution and make better informed choices of what needs to be build/provided and where.

* Select the best neighborhood to buy or rent your new home based on your venue priorities and preferences

## 2) Data <a name="data"></a>

Data is one of the most important resources a company can have in our current digital era, and yet everyone is currently swimming in a rising tide of data. This data lives in information systems, applications, devices and platforms, and it resides in the surrounding ecosystem – shared by suppliers, supply chain and distribution partners, investors, employees, end-use customers and consumers.

But it’s not just sheer volume. The sources and available types of data also continue to grow exponentially. In fact, 90 percent of the world’s current digital data was produced in the last two years, and the U.S. alone produces upward of 2.6 million gigabytes of internet data every minute.

It's important to make sure the data that  is being used to create statistical analysis and models reflect the reality of the situation. For that it is important to take in consideration a few characteristics:

* Accuracy and Precision
* Legitimacy and Validity
* Reliability and Consistency
* Timeliness and Relevance
* Completeness and Comprehensiveness
* Availability and Accessibility
* Granularity and Uniqueness

There are many elements that determine data quality, and each can be prioritized differently by different methods. The prioritization could change depending on the stage of growth of a project. 

The key is to remember you must define what is most important for your project when evaluating data. Then, use these characteristics to define the criteria for high-quality, accurate data. Once defined, you can be assured of a better understanding and are better positioned to achieve the desired goals.

### 2.1) Data Source

**The data that will be used in this project will come from two different sources:**

.

        1) New York University Libraries' portal for GIS data discovery
       
        2) Foursquare independent location data platform for Venues
        
.
        
        
Let's start importing our libraries and installing the necessary packages:

### 2.2) Import all necessary libraries:

In [1]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

! pip install geocoder
import geocoder
from geopy.geocoders import Nominatim 

import numpy as np

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import json

import requests # library to handle requests

# Matplotlib and associated plotting modules

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage

from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


### 2.3) Get Data: 

#### 2.3.1) NYU: New York city Neighborhoods, Postal Codes and Coordinates

Link: https://geo.nyu.edu/catalog/nyu_2451_34572

For this case the data was already downloaded in my hard drive.

We'll get the data, sort the values in alphabetical order and reset the index:

In [2]:
# read csv file
nyc_neighborhoods = pd.read_csv(r'C:\Users\lepet\Documents\Data Science\Coursera\IBM Data Science Professional\9 - Capstone\nyc_final.csv')

# sor
nyc_neighborhoods.sort_values(by='Neighborhood', axis=0, inplace=True)

nyc_neighborhoods = nyc_neighborhoods.set_index('Neighborhood').reset_index()

nyc_neighborhoods.shape , nyc_neighborhoods.head()

((306, 4),
     Neighborhood        Borough   Latitude  Longitude
 0       Allerton          Bronx  40.865788 -73.859319
 1       Annadale  Staten Island  40.538114 -74.178549
 2  Arden Heights  Staten Island  40.549286 -74.185887
 3      Arlington  Staten Island  40.635325 -74.165104
 4       Arrochar  Staten Island  40.596313 -74.067124)

As can be seen, the dataframe is composed by 306 Neighborhoods with the following attributes/columns:

* Borough
* Neighborhood
* Latitude
* Longitude

The coordinates will mainly be necessary to correlate with the Foursquare dataframe for each neighborhood venues.

#### 2.3.2) Foursquare: Venue data for each NY Neighborhood

Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below:

In [3]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


Input my own credentials for Foursquare requests:

In [4]:
CLIENT_ID = 'QZXHSZCTZZCQWWGKTPBCTVZRKOJVMRBBEACO3QTA3E3DLQ4E' # your Foursquare ID
CLIENT_SECRET = 'V30YXLRBSOYNV2OVCO5PEXDCZVMHCRC0CDBKBANIARTQKT2Y' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Foursquare API linked to your ID')

Foursquare API linked to your ID


Now that we have NYC location and our access to the Fourshare API, let's create a function to get the data from Foursquare servers

####  2.3.3) Create getNearbyVenues function

The function getNearbyVenues will get the following information:

* Venue Name 
* Venue Latitude
* Venue Longitude
* Venue Category

In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### 2.3.4) Aply the function in the nyc_neighborhoods coordinates

Now that the function is created, we can use it for each neighborhood in our dataframe and collect the first 100 Venues around it.

In [6]:
LIMIT = 100

nyc_venues = getNearbyVenues(names=nyc_neighborhoods['Neighborhood'],
                                       latitudes=nyc_neighborhoods['Latitude'],
                                       longitudes=nyc_neighborhoods['Longitude'],
                                )

Allerton
Annadale
Arden Heights
Arlington
Arrochar
Arverne
Astoria
Astoria Heights
Auburndale
Bath Beach
Battery Park City
Bay Ridge
Bay Terrace
Bay Terrace
Baychester
Bayside
Bayswater
Bedford Park
Bedford Stuyvesant
Beechhurst
Bellaire
Belle Harbor
Bellerose
Belmont
Bensonhurst
Bergen Beach
Blissville
Bloomfield
Boerum Hill
Borough Park
Breezy Point
Briarwood
Brighton Beach
Broad Channel
Broadway Junction
Bronxdale
Brooklyn Heights
Brookville
Brownsville
Bulls Head
Bushwick
Butler Manor
Cambria Heights
Canarsie
Carnegie Hill
Carroll Gardens
Castle Hill
Castleton Corners
Central Harlem
Charleston
Chelsea
Chelsea
Chinatown
City Island
City Line
Civic Center
Claremont Village
Clason Point
Clifton
Clinton
Clinton Hill
Co-op City
Cobble Hill
College Point
Concord
Concourse
Concourse Village
Coney Island
Corona
Country Club
Crown Heights
Cypress Hills
Ditmas Park
Dongan Hills
Douglaston
Downtown
Dumbo
Dyker Heights
East Elmhurst
East Flatbush
East Harlem
East New York
East Tremont
East Vil

In [7]:
nyc_venues.shape

(25039, 7)

Let's take a quick look at the top 20 venue categories:

In [8]:
nyc_ = nyc_venues['Venue Category'].value_counts()

nyc_top20 = nyc_.head(20)

nyc_top20

Pizza Place             1404
Italian Restaurant       741
Coffee Shop              673
Bakery                   596
Park                     588
Donut Shop               566
Sandwich Place           552
Bar                      502
Pharmacy                 498
Grocery Store            494
Ice Cream Shop           475
Deli / Bodega            455
Mexican Restaurant       431
Bank                     416
Caribbean Restaurant     390
Chinese Restaurant       379
Gym                      352
Fast Food Restaurant     335
Bagel Shop               329
American Restaurant      322
Name: Venue Category, dtype: int64

As we can see, the top 20 venue categories represents about 41% of the total amount of venues types, and as we expected, Pizza Places are the most common location, followed by Italian Restaurants and Coffee shops.

Now that we have all necessary data, we can start modeling it to achieve our objective.

## 3) Methodology <a name="methodology"></a>

### 3.1) Introduction

In this project we will direct our efforts on detecting similarities in New York city neighborhoods that aren't visible if you just look at their physical location, we want to discover what neighborhoods are similar to others based on the venue types, not necessarily neighborhoods that are close to each other.

In first step we have collected the required information for the exploratory data analysis (nyc_neighborhoods and nyc_venues). We have limited our analysis to a area ~1.5km around each neighborhood, so that we can get enough venue categories for each neighborhood, improving the accuracy of the model.

The second step in our project is to prepare the data that we'll use to categorize our neighborhoods, as well as creating the machine learning model to cluster them together (using **k-means clustering**). We will finally present a map of all such locations and their respective cluster, identifying  how they were connected collectively.

In third and final step we will focus on describing and discussing the results of the clustering model created, understanding  why some neighborhoods were defined as they were, what the final outcome can tell us about New York's community and surroundings, as well as wrapping  up the project with a final conclusion of the work done. 

Let's start with the main variable that will be used for this analysis: the Venue Category, that will be the basis to understand, sort and classify each neighborhood.

### 3.2) Encoding Categorical Values in "Venue Category"

Because machine learning models require all input and output variables to be numeric, we first need to transform the Venue Category from categorical to quantitative. One great way to do that is through the one-hot encoding, where each bit represents a possible category. If the variable cannot belong to multiple categories at once, then only one bit in the group can be “on”. 

We'll use the "pandas.get_dummies" function to perfom this transformation:

In [9]:
# one hot encoding
onehot = pd.get_dummies(nyc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = nyc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

#sort values alphabetically 
onehot = onehot.sort_values(by ='Neighborhood' )

#reset the index for the new dataframe
onehot = onehot.set_index('Neighborhood').reset_index()

onehot.head()

,Neighborhood,Zoo Exhibit,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Service,Airport Terminal,Airport Tram,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Casino,Caucasian Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Gym,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Kofte Place,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving

Now that all the values are numerical, let's understand the size of the new dataframe

In [10]:
onehot.shape

(25039, 457)

As we can see, there are 451 types of venues in this dataframe and they'll be the basis to cluster the neighborhoods together.

Let's also group all the neighborhoods rows so that all their variables are represented in just one row of data.

In [11]:
grouped = onehot.groupby('Neighborhood').sum().reset_index()
grouped.head(10)

,Neighborhood,Zoo Exhibit,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Service,Airport Terminal,Airport Tram,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Casino,Caucasian Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Gym,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Kofte Place,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving

In [12]:
grouped.shape

(302, 457)

With the grouped dataframe created we can finally start clustering the neighborhoods base on their venues.

### 3.3) Clustering

A cluster is a group of data points or objects in a dataset that are similar to other objects in the group, and dissimilar to datapoints in other clusters.  In clustering the data is unlabeled and the process is unsupervised.

Cluster analysis is an exploratory analysis that tries to identify structures within the data.  Cluster analysis is also called segmentation analysis or taxonomy analysis.  More specifically, it tries to identify homogenous groups of cases if the grouping is not previously known.  Because it is exploratory, it does not make any distinction between dependent and independent variables.

We'll use a K-means, a Particioned-based Clustering algorithm, that the main objective is to minimize the distance of data points from the centroid of its cluster and maximize the distance from other cluster centroids.

We'll use the Scikit-Learn library to train the model as follows:

In [13]:
#Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 10

#only numerical data in for the training
grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=5).fit(grouped_clustering)

print('Training Complete!')

Training Complete!


In [15]:
#drop values
#grouped.drop(['Cluster Labels'], axis=1, inplace = True)

# add clustering labels
grouped.insert(0, 'Cluster Labels', kmeans.labels_)

merged = grouped

# merge nyc_neighborhoods to merged to add latitude/longitude for each neighborhood
merged = merged.join(nyc_neighborhoods.set_index('Neighborhood'), on='Neighborhood')

merged.head(3)

,Cluster Labels,Neighborhood,Zoo Exhibit,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Service,Airport Terminal,Airport Tram,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Casino,Caucasian Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Gym,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Kofte Place,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie

### 3.4) Clustering Visualization

Data visualization is the graphical representation of information and data. By using visual elements like charts, graphs, and maps, data visualization tools provide an accessible way to see and understand trends, outliers, and patterns in data.

In the world of Big Data, data visualization tools and technologies are essential to analyze massive amounts of information and make data-driven decisions.

#### 3.4.1) Folium Maps

Folium is a powerful Python library that helps you create several types of Leaflet maps. The fact that the Folium results are interactive makes this library very useful for dashboard building.

Folium makes it easy to visualize data that’s been manipulated in Python on an interactive leaflet map. It enables both the binding of data to a map for choropleth visualizations as well as passing rich vector/raster/HTML visualizations as markers on the map.

In [16]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We can see that the visualization of the clustering helps a lot to understand how the neighborhoods were clustered, communicating findings in constructive ways.

This concludes the data exploration, so lets start discussing the results of these clusters and why they were categorized this way in the Results and Discussion section bellow.

## 4) Results and Discussion <a name="results"></a>

The Results section of this project represents the core findings of the study derived from the methods applied to gather and analyze information. It presents these findings in a logical sequence without bias or interpretation, setting up the reader for  interpretation and evaluation of the data. 

A major purpose of the results section is to break down the data into sentences that show its significance to the research question, so lets dig deeper into what neighborhoods are similar to others based on the venue types, not necessarily neighborhoods that are close to eachother.

### 4.1) Understanding each cluster

Let's first understand the size of each group:

In [17]:
merged['Cluster Labels'].value_counts(sort=False)

0    16
1    12
2    56
3    50
4    15
5    43
6    30
7     3
8     6
9    75
Name: Cluster Labels, dtype: int64

We can see that only 2 clusters have a lower amount of items and the other 8 have a reasonable size. 

Let's take a deeper look in the contents of each Cluster:

In [18]:
clustered_neighborhoods = grouped.loc[: , ['Cluster Labels', 'Neighborhood']]

clustered_neighborhoods.sort_values(by = ['Cluster Labels', 'Neighborhood'], ascending=True, inplace=True)

clustered_neighborhoods.reset_index(inplace=True, drop=True)

clustered_neighborhoods

,Cluster Labels,Neighborhood
0,0,Bedford Park
1,0,Borough Park
2,0,Bronxdale
3,0,Kingsbridge
4,0,Kingsbridge Heights
5,0,Marble Hill
6,0,Morris Park
7,0,Parkchester
8,0,Pelham Parkway
9,0,Riverdale


The clusters are mainly composed by neighborhoods that are close to eachother but you can definetly see some items that can be considered outliers by their coordinate disparity.

This means that a neighborhood can be physicaly far away from their group but it is composed by the same characteristics and venue categories than the other elements inside the same group.

### 4.2) Understanding each cluster essential venue categories

Let's create a new dataframe with the top 10 most common venues for each cluster:

In [19]:
grouped2 = grouped

grouped_cluster = grouped2.groupby(by=['Cluster Labels']).sum()

grouped_cluster.reset_index(inplace=True)

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues

columns = ['Cluster Labels']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Cluster Labels'] = grouped_cluster['Cluster Labels']

for ind in np.arange(grouped_cluster.shape[0]):
    
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped_cluster.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.set_index('Cluster Labels', inplace=True)
    
neighborhoods_venues_sorted.head(10)

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,
0,Pizza Place,Mexican Restaurant,Coffee Shop,Donut Shop,Diner,Sandwich Place,Park,Bar,Latin American Restaurant,Gym
1,Clothing Store,Pizza Place,Pharmacy,Caribbean Restaurant,Fast Food Restaurant,Donut Shop,Department Store,Sandwich Place,Cosmetics Shop,Discount Store
2,Pizza Place,Italian Restaurant,Bank,Pharmacy,Bakery,Donut Shop,Ice Cream Shop,Sandwich Place,Bagel Shop,Deli / Bodega
3,Coffee Shop,Bar,Pizza Place,Bakery,Italian Restaurant,Mexican Restaurant,Cocktail Bar,Café,Thai Restaurant,Yoga Studio
4,Caribbean Restaurant,Pizza Place,Bakery,Café,Grocery Store,Pharmacy,Donut Shop,Deli / Bodega,Discount Store,Ice Cream Shop
5,Pizza Place,Donut Shop,Pharmacy,Sandwich Place,Fast Food Restaurant,Discount Store,Grocery Store,Bank,Caribbean Restaurant,Supermarket
6,Park,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Bakery,American Restaurant,Gym,Pizza Place,Hotel,Theater
7,Korean Restaurant,Coffee Shop,Pizza Place,Grocery Store,Bakery,Gym / Fitness Center,Japanese Restaurant,Chinese Restaurant,Indian Restaurant,Ice Cream Shop
8,Beach,Pizza Place,Donut Shop,Bar,Deli / Bodega,Bagel Shop,Surf Spot,Pharmacy,Board Shop,Ice Cream Shop


 The dataframe above provides a clear view of why these clusters are created and why each element belongs together. 

Other main results that are important to highlight:

* "Pizza Place" is in all clusters top 10 common venues
* Cluster 1 main distinctive features are "Clothing Store" and "Pharmacy"
* Cluster 7 has a very wider Asian cuisine influence
* Cluster 8 main distinctive features are "Beach" and "Donut Shop"
* Although "Italian Restaurant" is the second most common item, it is only in 4 out of 10 cluster top 10 common venues, meaning that they might be grouped together instead of being spread all over New York
* Cluster 6 main distinctive features are "Park" and "Coffee Shop"

## 5) Conclusion <a name="conclusion"></a>

This project had the objective of identifying connections in New York city neighborhoods that aren't evident if only their actual location and coordinates are taken in consideration and explore why those new connections were made.

We initially gathered and cleaned the necessary data from New York University Libraries' portal and Foursquare independent location data platform.

The data was later encoded and clustered bases on their venue characteristics, 10 clusters were created with a total of 302 neighborhoods.

Using Folium as our data visualization tool was possible to analyse their affinities, closeness and correlation. 

The results shown is this project can and should be used for many diffent other diffent purposes of data classification for better decision making.